In [1]:
import geopandas as gpd
import matplotlib as plt
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from shapely.geometry import Point
6
%matplotlib inline

import seaborn as sns
plt.style.use('seaborn')
plt.rcParams['figure.dpi'] = 160

In [2]:
fire_data=pd.read_csv('./data/basicincident.txt')

ParserError: Error tokenizing data. C error: Expected 1 fields in line 9538864, saw 2


In [4]:
pd.options.display.max_columns = None

In [5]:
fire_data.head()

,Call Number,Unit ID,Incident Number,Call Type,Call Date,Watch Date,Received DtTm,Entry DtTm,Dispatch DtTm,Response DtTm,On Scene DtTm,Transport DtTm,Hospital DtTm,Call Final Disposition,Available DtTm,Address,City,Zipcode of Incident,Battalion,Station Area,Box,Original Priority,Priority,Final Priority,ALS Unit,Call Type Group,Number of Alarms,Unit Type,Unit sequence in call dispatch,Fire Prevention District,Supervisor District,Neighborhooods - Analysis Boundaries,RowID,case_location,Analysis Neighborhoods
0,221210313,E36,22054955,Outside Fire,05/01/2022,04/30/2022,05/01/2022 02:58:25 AM,05/01/2022 02:59:15 AM,05/01/2022 02:59:25 AM,05/01/2022 03:01:06 AM,05/01/2022 03:02:27 AM,NaN,NaN,Fire,05/01/2022 03:05:00 AM,GOUGH ST/GROVE ST,San Francisco,94102.0,B02,36.0,3265,3,3,3,True,Fire,1,ENGINE,1.0,2,5,Hayes Valley,221210313-E36,POINT (-122.42316555403964 37.77781524520032),9.0
1,220190150,E29,22008871,Alarms,01/19/2022,01/18/2022,01/19/2022 01:42:12 AM,01/19/2022 01:44:13 AM,01/19/2022 01:44:28 AM,01/19/2022 01:46:47 AM,01/19/2022 01:49:32 AM,NaN,NaN,Fire,01/19/2022 02:35:26 AM,100 Block of MISSISSIPPI ST,San Francisco,94107.0,B03,29.0,2431,3,3,3,True,Alarm,1,ENGINE,1.0,3,10,Potrero Hill,220190150-E29,POINT (-122.39469970274361 37.76460987856451),26.0
2,211233271,T07,21053032,Alarms,05/03/2021,05/03/2021,05/03/2021 09:28:12 PM,05/03/2021 09:28:12 PM,05/03/2021 09:28:17 PM,05/03/2021 09:29:10 PM,05/03/2021 09:32:15 PM,NaN,NaN,Fire,05/03/2021 09:38:09 PM,0 Block of HOFF ST,San Francisco,94110.0,B02,7.0,5236,A,3,3,False,Alarm,1,TRUCK,2.0,2,9,Mission,211233271-T07,POINT (-122.42057572093252 37.76418194637148),20.0
3,212933533,B02,21127914,Alarms,10/20/2021,10/20/2021,10/20/2021 10:08:47 PM,10/20/2021 10:09:53 PM,10/20/2021 10:10:07 PM,10/20/2021 10:11:55 PM,NaN,NaN,NaN,Fire,10/20/2021 10:25:52 PM,200 Block of JONES ST,San Francisco,94102.0,B03,3.0,1456,3,3,3,False,Alarm,1,CHIEF,3.0,3,6,Tenderloin,212933533-B02,POINT (-122.41243514072728 37.78347684038771),36.0
4,221202543,E41,22054815,Alarms,04/30/2022,04/30/2022,04/30/2022 06:35:58 PM,04/30/2022 06:37:28 PM,04/30/2022 06:37:43 PM,04/30/2022 06:38:17 PM,NaN,NaN,NaN,Fire,04/30/2022 06:40:08 PM,1400 Block of FILBERT ST,San Francisco,94109.0,B04,16.0,3146,3,3,3,False,Alarm,1,ENGINE,4.0,4,2,Russian Hill,221202543-E41,POINT (-122.4233369425531 37.799534868680034),32.0


In [6]:
fire_data['Call Type'].unique()

array(['Outside Fire', 'Alarms', 'Structure Fire', 'Medical Incident',
       'Other', 'Citizen Assist / Service Call', 'Traffic Collision',
       'Electrical Hazard', 'Water Rescue', 'Elevator / Escalator Rescue',
       'Fuel Spill', 'High Angle Rescue', 'Odor (Strange / Unknown)',
       'Gas Leak (Natural and LP Gases)', 'Vehicle Fire', 'Explosion',
       'Smoke Investigation (Outside)', 'HazMat', 'Assist Police',
       'Extrication / Entrapped (Machinery, Vehicle)',
       'Confined Space / Structure Collapse', 'Watercraft in Distress',
       'Mutual Aid / Assist Outside Agency', 'Suspicious Package',
       'Industrial Accidents', 'Structure Fire / Smoke in Building',
       'Administrative', 'Marine Fire', 'Train / Rail Incident',
       'Oil Spill', 'Train / Rail Fire', 'Aircraft Emergency',
       'Lightning Strike (Investigation)'], dtype=object)

In [7]:
fire_data[fire_data['Call Type']=='Vehicle Fire']

,Call Number,Unit ID,Incident Number,Call Type,Call Date,Watch Date,Received DtTm,Entry DtTm,Dispatch DtTm,Response DtTm,On Scene DtTm,Transport DtTm,Hospital DtTm,Call Final Disposition,Available DtTm,Address,City,Zipcode of Incident,Battalion,Station Area,Box,Original Priority,Priority,Final Priority,ALS Unit,Call Type Group,Number of Alarms,Unit Type,Unit sequence in call dispatch,Fire Prevention District,Supervisor District,Neighborhooods - Analysis Boundaries,RowID,case_location,Analysis Neighborhoods
642,211101255,T05,21047420,Vehicle Fire,04/20/2021,04/20/2021,04/20/2021 11:07:58 AM,04/20/2021 11:13:07 AM,04/20/2021 11:13:28 AM,04/20/2021 11:13:35 AM,04/20/2021 11:16:23 AM,NaN,NaN,No Merit,04/20/2021 11:17:05 AM,TURK ST/FILLMORE ST,San Francisco,94115.0,B05,5.0,3535,3,3,3,False,Fire,1,TRUCK,1.0,5,5,Western Addition,211101255-T05,POINT (-122.43214039082407 37.78049625961475),39.0
643,211101255,E05,21047420,Vehicle Fire,04/20/2021,04/20/2021,04/20/2021 11:07:58 AM,04/20/2021 11:13:07 AM,04/20/2021 11:13:28 AM,04/20/2021 11:13:44 AM,04/20/2021 11:17:52 AM,NaN,NaN,No Merit,04/20/2021 11:17:58 AM,TURK ST/FILLMORE ST,San Francisco,94115.0,B05,5.0,3535,3,3,3,True,Fire,1,ENGINE,2.0,5,5,Western Addition,211101255-E05,POINT (-122.43214039082407 37.78049625961475),39.0
1256,211811825,E15,21077578,Vehicle Fire,06/30/2021,06/30/2021,06/30/2021 01:51:02 PM,06/30/2021 01:51:51 PM,06/30/2021 01:52:02 PM,06/30/2021 01:53:44 PM,06/30/2021 01:54:22 PM,NaN,NaN,Fire,06/30/2021 02:22:03 PM,800 Block of OCEAN AV,San Francisco,94112.0,B09,15.0,8472,3,3,3,True,Fire,1,ENGINE,1.0,9.0,7,Oceanview/Merced/Ingleside,211811825-E15,POINT (-122.45058936484298 37.722898665635554),24.0
1257,211811825,T15,21077578,Vehicle Fire,06/30/2021,06/30/2021,06/30/2021 01:51:02 PM,06/30/2021 01:51:51 PM,06/30/2021 01:52:02 PM,06/30/2021 01:53:22 PM,06/30/2021 02:10:28 PM,NaN,NaN,Fire,06/30/2021 02:15:21 PM,800 Block of OCEAN AV,San Francisco,94112.0,B09,15.0,8472,3,3,3,False,Fire,1,TRUCK,2.0,9.0,7,Oceanview/Merced/Ingleside,211811825-T15,POINT (-122.45058936484298 37.722898665635554),24.0
2660,212801761,T07,21121754,Vehicle Fire,10/07/2021,10/07/2021,10/07/2021 01:30:39 PM,10/07/2021 01:31:00 PM,10/07/2021 01:31:09 PM,10/07/2021 01:32:00 PM,10/07/2021 01:33:33 PM,NaN,NaN,Fire,10/07/2021 01:43:14 PM,400 Block of TREAT AVE,San Francisco,94110.0,B02,7.0,5245,3,3,3,False,Fire,1,TRUCK,2.0,2,9,Mission,212801761-T07,POINT (-122.41387736611333 37.76300174054981),20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6375020,232840116,T18,23137016,Vehicle Fire,10/11/2023,10/10/2023,10/11/2023 01:22:33 AM,10/11/2023 01:22:33 AM,10/11/2023 01:22:39 AM,10/11/2023 01:23:59 AM,10/11/2023 01:26:48 AM,NaN,NaN,Fire,10/11/2023 01:42:03 AM,SUNSET BL/NORIEGA ST,San Francisco,94122.0,B08,18,7562,3,3,3,False,Fire,1,TRUCK,1.0,8,4,Sunset/Parkside,232840116-T18,POINT (-122.49522653812167 37.75341381749554),35.0
6375049,232840017,E23,23136998,Vehicle Fire,10/11/2023,10/10/2023,10/11/2023 12:08:29 AM,10/11/2023 12:09:26 AM,10/11/2023 12:10:20 AM,10/11/2023 12:11:49 AM,10/11/2023 12:13:35 AM,NaN,NaN,Fire,10/11/2023 12:44:21 AM,KIRKHAM ST/46TH AV,San Francisco,94122.0,B08,23,7717,3,3,3,True,Fire,1,ENGINE,1.0,8,4,Sunset/Parkside,232840017-E23,POINT (-122.50579684716125 37.758559525047936),35.0
6377234,232830337,T09,23136589,Vehicle Fire,10/10/2023,10/09/2023,10/10/2023 05:49:26 AM,10/10/2023 05:50:46 AM,10/10/2023 05:51:00 AM,10/10/2023 05:54:08 AM,10/10/2023 05:59:45 AM,NaN,NaN,Fire,10/10/2023 06:04:50 AM,100 Block of CARGO WAY,San Francisco,94124.0,B10,25,6552,3,3,3,False,Fire,1,TRUCK,1.0,10,10,Bayview Hunters Point,232830337-T09,POINT (-122.37993166984897 37.74209199934995),1.0
6377924,232962209,E16,23143233,Vehicle Fire,10/23/2023,10/23/2023,10/23/2023 04:32:06 PM,10/23/2023 04:34:25 PM,10/23/2023 04:34:49 PM,10/23/2023 04:36:01 PM,10/23/2023 04:38:37 PM,NaN,NaN,Fire,10/23/2023 04:41:13 PM,1600 Bloc